# Retraction System Smoke Test

This notebook verifies the retraction system is working correctly:
1. Alembic migrations applied
2. Database schema has retraction columns
3. Retraction affects event counts


In [ ]:
# Cell 1: Check Alembic State
import subprocess
import os

os.chdir('../..')
os.chdir('infra/migrations')

result = subprocess.run(["python3", "-m", "alembic", "current"], capture_output=True, text=True)
print("Current migration:")
print(result.stdout)

result = subprocess.run(["python3", "-m", "alembic", "heads"], capture_output=True, text=True)
print("\nAlembic heads:")
print(result.stdout)

result = subprocess.run(["python3", "-m", "alembic", "branches"], capture_output=True, text=True)
print("\nBranches:")
print(result.stdout if result.stdout.strip() else "No branches (good!)")

os.chdir('../..')


In [ ]:
# Cell 2: Verify Schema
from sqlalchemy import create_engine, text
import os

database_url = os.environ.get("DATABASE_URL")
if not database_url:
    print("⚠️  DATABASE_URL not set. Using default.")
    database_url = "postgresql+psycopg://localhost/agi_tracker"

engine = create_engine(database_url)
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT column_name, data_type, is_nullable
        FROM information_schema.columns 
        WHERE table_name = 'events' 
        AND column_name IN ('retracted_at', 'retraction_reason', 'retraction_evidence_url')
        ORDER BY column_name
    """))
    print("Retraction columns in events table:")
    print("-" * 60)
    for row in result:
        print(f"  {row.column_name:30s} {row.data_type:20s} nullable={row.is_nullable}")
    
    # Check indexes
    result = conn.execute(text("""
        SELECT indexname, indexdef
        FROM pg_indexes
        WHERE tablename = 'events'
        AND (indexname LIKE '%retract%' OR indexdef LIKE '%retract%')
    """))
    print("\nRetraction-related indexes:")
    print("-" * 60)
    for row in result:
        print(f"  {row.indexname}")
        print(f"    {row.indexdef}")


In [ ]:
# Cell 3: Test Retraction Effect
import sys
sys.path.insert(0, 'services/etl')

from app.database import SessionLocal
from app.models import Event
from datetime import datetime, timezone

db = SessionLocal()

# Count before
before = db.query(Event).filter(Event.retracted == False).count()
total = db.query(Event).count()
print(f"Active events before: {before}")
print(f"Total events: {total}")
print(f"Retracted events: {total - before}")

# Find a test event (or create one)
test_event = db.query(Event).filter(Event.retracted == False).first()
if test_event:
    print(f"\nTest event: {test_event.id} - {test_event.title[:50]}")
    original_retracted = test_event.retracted
    
    # Temporarily retract
    test_event.retracted = True
    test_event.retracted_at = datetime.now(timezone.utc)
    test_event.retraction_reason = "Smoke test - will be reverted"
    db.commit()
    
    # Count after
    after = db.query(Event).filter(Event.retracted == False).count()
    print(f"\nActive events after retraction: {after}")
    print(f"Difference: {before - after} (should be 1)")
    
    # Rollback
    test_event.retracted = original_retracted
    test_event.retracted_at = None
    test_event.retraction_reason = None
    db.commit()
    
    # Verify rollback
    final = db.query(Event).filter(Event.retracted == False).count()
    print(f"\nActive events after rollback: {final}")
    print(f"Matches original: {final == before}")
    
    if before - after == 1 and final == before:
        print("\n✅ Smoke test PASSED")
    else:
        print("\n❌ Smoke test FAILED")
else:
    print("\n⚠️  No events found to test")

db.close()


## Summary

If all cells executed successfully:
- ✅ Alembic migrations are applied
- ✅ Retraction columns exist in database
- ✅ Retraction affects event counts correctly

The retraction system is operational!
